<a href="https://colab.research.google.com/github/x110/DLToolboxImg/blob/master/NoduleSegmentationModel8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configuration

In [0]:
# Remove CUDA 9 completely

!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove

!apt-get update

In [0]:
# Install CUDA 8

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!dpkg -i --force-overwrite cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!apt-get update
!apt-get install cuda-8-0

# install will fail, need to force dpkg to overwrite the configuration file

!wget http://archive.ubuntu.com/ubuntu/pool/main/m/mesa/libglx-mesa0_18.0.5-0ubuntu0~18.04.1_amd64.deb
!dpkg -i --force-overwrite libglx-mesa0_18.0.5-0ubuntu0~18.04.1_amd64.deb

!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/nvidia-410_410.48-0ubuntu1_amd64.deb
!dpkg -i --force-overwrite nvidia-410_410.48-0ubuntu1_amd64.deb

!apt --fix-broken install
!apt-get install cuda-8-0

In [0]:
!pip uninstall -y numpy
!pip uninstall -y scipy
!pip install turicreate
# The worng version of MXNET will be installed.
!pip uninstall -y mxnet
!pip install scipy
# Instal CUDA8-compatible version of mxnet 1.1.0
!pip install mxnet-cu80==1.1.0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Settings

In [0]:
dataset_path = '/content/drive/My Drive/Datasets/Dataset5/processed/.'
model_path = "drive/My Drive/x110/model/model8/"
#model details
prefix = "nov8"
model_epoch=40

In [0]:
epochs = 10
load_model = True#False
find_stats = False#True
bs = 34#100#64#34

#  Load Data

In [0]:
PATH = "x110/"

In [4]:
!mkdir {PATH}
!mkdir {PATH+"Datasets"}
!mkdir {PATH+"Datasets/Dataset1/"}
!mkdir {PATH+"Datasets/Dataset1/processed/"}
!mkdir {PATH+"model"}

mkdir: cannot create directory ‘x110/’: File exists
mkdir: cannot create directory ‘x110/Datasets’: File exists
mkdir: cannot create directory ‘x110/Datasets/Dataset1/’: File exists
mkdir: cannot create directory ‘x110/Datasets/Dataset1/processed/’: File exists
mkdir: cannot create directory ‘x110/model’: File exists


In [0]:
!cp -a "{dataset_path}" {PATH+"Datasets/Dataset1/processed/"}

In [0]:
if load_model:
  drivefile = model_path+prefix+"-"+"{:04d}".format(model_epoch)+".params"
  drivefile2 = model_path+prefix+"-symbol.json"
  colabfile = PATH+"model/"+prefix+"-"+"{:04d}".format(model_epoch)+".params"
  colabfile2 = PATH+"model/"+prefix+"-symbol.json"

  !cp  "{drivefile}" {colabfile}
  !cp  "{drivefile2}" {colabfile2}


In [0]:
#Dataset Path 
interm_dir4 = PATH + "Datasets/Dataset1/processed/"
s = "2018_11_25"
train_data_path=interm_dir4+'train'+s+'pos.rec'
train_idx_path=interm_dir4+'train'+s+'pos.idx'
valid_data_path=interm_dir4+'valid'+s+'pos.rec'
valid_idx_path=interm_dir4+'valid'+s+'pos.idx'
test_data_path=interm_dir4+'test'+s+'pos.rec'
test_idx_path=interm_dir4+'test'+s+'pos.idx'

In [8]:
!mkdir x110/DLToolboxImg
!git clone https://github.com/x110/DLToolboxImg.git x110/DLToolboxImg

mkdir: cannot create directory ‘x110/DLToolboxImg’: File exists
fatal: destination path 'x110/DLToolboxImg' already exists and is not an empty directory.


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configuration" data-toc-modified-id="Configuration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Find-mean-of-images" data-toc-modified-id="Find-mean-of-images-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Find mean of images</a></span></li><li><span><a href="#Find-variance-of-images" data-toc-modified-id="Find-variance-of-images-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Find variance of images</a></span></li></ul></li><li><span><a href="#Data-Iterator" data-toc-modified-id="Data-Iterator-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Iterator</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluation Metric</a></span></li><li><span><a href="#Model-Architecture" data-toc-modified-id="Model-Architecture-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model Architecture</a></span></li><li><span><a href="#Optimizer" data-toc-modified-id="Optimizer-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Optimizer</a></span><ul class="toc-item"><li><span><a href="#Find--learning-rate" data-toc-modified-id="Find--learning-rate-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Find  learning rate</a></span></li><li><span><a href="#Optimizer-Parameters" data-toc-modified-id="Optimizer-Parameters-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Optimizer Parameters</a></span></li></ul></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Train Model</a></span></li><li><span><a href="#Evaluate-Model" data-toc-modified-id="Evaluate-Model-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Evaluate Model</a></span></li></ul></div>

# Model Code

In [0]:
import sys
sys.path.insert(0,"x110/DLToolboxImg/src/")

In [0]:
import mynnet7 as nn
import pickle
import mxnet as mx
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Nodule Segmentation: Model

## Preprocessing

### Find mean of images

In [12]:
if find_stats:
    BATCH_SIZE=1
    train_iter=nn.FileIter(train_data_path,train_idx_path,batch_size=BATCH_SIZE,do_augment=True,mean_image=0,std_image=1)
    train_iter.reset()
    x_mean = np.zeros((32,32,32))
    for i,batch in enumerate(train_iter):
        X =  batch.data[0][0][0].asnumpy()
        x_mean+=X
    x_mean=np.mean(x_mean/i)
    # Saving the objects:
    with open(interm_dir4+'x_mean.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([x_mean], f)

else:
    with open(interm_dir4+'x_mean.pkl','rb') as f:  # Python 3: open(..., 'rb')
        x_mean = pickle.load(f)
x_mean#x_mean=0.2826227159416579

[0.28262271594165789]

In [0]:
x_mean=0.28

### Find variance of images

In [14]:
if find_stats: 
    BATCH_SIZE=1
    train_iter=nn.FileIter(train_data_path,train_idx_path,batch_size=BATCH_SIZE,do_augment=True,mean_image=0,std_image = 1)
    train_iter.reset()
    x_var = np.zeros((32,32,32))
    for i,batch in enumerate(train_iter):
        X =  (batch.data[0][0][0].asnumpy()-x_mean)**2
        x_var+=X
    #x_var=x_var/(i-1)
    #x_var#x_mean=.2815
    N = i*32*32*32
    x_var = np.sum(x_var)/(N-1)
    x_var#x_var = .07877
    x_std = np.sqrt(x_var)#x_std=.2807
    with open(interm_dir4+'x_std.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([x_std], f)
else:
    with open(interm_dir4+'x_std.pkl','rb') as f:  # Python 3: open(..., 'rb')
        x_std = pickle.load(f)
x_std#x_std=0.2817920662435274

[0.28179206624352737]

In [0]:
x_std = 0.28

## Data Iterator

In [16]:
BATCH_SIZE=bs
train_iter=nn.FileIter(train_data_path,train_idx_path,batch_size=BATCH_SIZE,random_flip=True,do_augment=False,mean_image=x_mean,std_image = x_std,do_shuffle=True)
input_shapes = dict(train_iter.provide_data+train_iter.provide_label)
print(input_shapes)

{'data': (34, 1, 32, 32, 32), 'softmax_label': (34, 32768)}


In [0]:
BATCH_SIZE=bs
valid_iter=nn.FileIter(valid_data_path,valid_idx_path,batch_size=BATCH_SIZE,do_augment=False,mean_image=x_mean,std_image = x_std,do_shuffle=False)

In [0]:
train_iter.reset()
valid_iter.reset()

## Evaluation Metric

In [0]:
def dice_coef2(label, y):
    smooth = 1.
    label=mx.nd.array(label).as_in_context(mx.gpu(0))
    y=mx.nd.array(y).as_in_context(mx.gpu(0))
    intersection = mx.nd.sum(label*y)
    return ((2. * intersection + smooth) / (mx.nd.sum(label) +mx.nd.sum(mx.nd.abs(y)) + smooth))


In [0]:
def logloss2(label, y,w=[.9,.1]):
    smooth = 1.
    label=mx.nd.array(label).as_in_context(mx.gpu(0))
    y=mx.nd.array(y).as_in_context(mx.gpu(0))
    eps=1e-12
    return mx.nd.mean(-(w[0]*label*mx.nd.log(y+eps)+w[1]*(1-label)*mx.nd.log(1-y+eps)))

In [0]:
# ===============Evaluation metric(s)================= 
eval_metrics_1 =  mx.metric.CustomMetric(feval=logloss2)
eval_metrics_2 = mx.metric.CustomMetric(feval=nn.dice_coef2)
metric= mx.metric.CompositeEvalMetric()
for child_metric in [eval_metrics_1, eval_metrics_2]:
  metric.add(child_metric)

## Model Architecture

In [22]:
load_model

True

In [0]:
if load_model:
    network, arg_params, aux_params = mx.model.load_checkpoint(model_path+prefix , model_epoch)

    # Binding
    exe = network.simple_bind(ctx=mx.gpu(0), **input_shapes)


    exe.copy_params_from(arg_params, aux_params)
    
    # get handle to input arrays
    arg_arrays = dict(zip(network.list_arguments(), exe.arg_arrays))
    data = arg_arrays[train_iter.provide_data[0][0]]
    label = arg_arrays[train_iter.provide_label[0][0]]
else:

    network = nn.get_net_319()
    init = mx.init.Normal(0.01) #note biases and gamma/beta are not affected

    # Binding
    exe = network.simple_bind(ctx=mx.gpu(), **input_shapes)
    # get handle to input arrays
    arg_arrays = dict(zip(network.list_arguments(), exe.arg_arrays))
    data = arg_arrays[train_iter.provide_data[0][0]]
    label = arg_arrays[train_iter.provide_label[0][0]]
    for name, arr in arg_arrays.items():
        if name not in input_shapes:
            init(name, arr)


# Nodule Segmentation: Model

## Optimizer

### Find  learning rate

In [142]:
find_learning_rate = True

nb=train_iter.num_data//train_iter.batch_size

if find_learning_rate:

  lrs = []
  loss = []
  
  sched=nn.lr_find(1e-5,nb,end_lr=1e-4)
  # We also need to create an optimizer for updating weights
  opt = mx.optimizer.SGD(
      learning_rate=1e-5,
      momentum=0.9,
      wd=0.00001,
      lr_scheduler=sched)

  updater = mx.optimizer.get_updater(opt)
  for epoch in range(0,1):

      start = time.time()

      metric.reset()

      train_iter.reset()

      valid_iter.reset()

      sched.reset()

      sched.on_train_begin()


      for batch in train_iter:
          # Copy data to executor input. Note the [:].
          data[:] = batch.data[0]
          label[:] = batch.label[0]

          # Forward
          outputs=exe.forward(is_train=True)
          # Backward
          exe.backward()

          # Update
          for i, pair in enumerate(zip(exe.arg_arrays, exe.grad_arrays)):
              weight, grad = pair
              updater(i, grad, weight)   
          #metric.update(batch.label[0], exe.outputs[0])#
          metric.update(batch.label[0], exe.outputs[0])

          e=metric.get()
          e = dict(zip(e[0], e[1]))
  
          err_train=-e['dice_coef2'].asnumpy()[0]
          sched.on_batch_end(err_train)

      if epoch % 100== 0:       
          #print("do_checkpoint")
          arg={k:v for k, v in arg_arrays.items() if k not in input_shapes}
          aux = dict(zip(network.list_auxiliary_states(), exe.aux_arrays))
          mx.model.save_checkpoint(prefix, epoch, network, arg, aux)


      #compute valid loss per epoch    
      metric.reset()
      for batch in valid_iter:        
          data[:] = batch.data[0]       
          label[:] = batch.label[0]
          # predict
          outputs = exe.forward(is_train=False)
          metric.update(batch.label[0], exe.outputs[0])
      e0=metric.get()
      e = dict(zip(e0[0], e0[1]))
      err_valid=-e['dice_coef2'].asnumpy()[0]
      end = time.time()
      print('time:',end-start,'Epoch:',epoch,'trainloss:',err_train,'validloss:',err_valid,'CE',outputs[1].asnumpy())
      for x,y in zip(e0[0],e0[1]):
        print(x,y[0].asnumpy()[0], end=", ", flush=True)
  lrs.extend(sched.lrs)
  loss.extend(sched.losses)



time: 136.57926440238953 Epoch: 0 trainloss: -0.700098 validloss: -0.627427 CE [-0.68972367]
logloss2 0.272537, dice_coef2 0.627427, 

In [0]:
  plt.figure()
  plt.plot(lrs, loss,'*-')

### Optimizer Parameters

In [0]:
#logfile = model_path+"errorlogmodel8.csv"
logfile = "errorlogmodel8.csv"

In [0]:

col_names = ['epoch',
             'lr',
             'loss',
             'E1train',
             'E1valid',
             'E2train',
             'E2valid']
df = pd.DataFrame(columns=col_names)

if False:
  df.to_csv(logfile)

In [128]:
df = pd.read_csv(logfile, index_col=0)
df.head()

,epoch,lr,loss,E1train,E1valid,E2train,E2valid


In [0]:
epochs = 10
lr =1.5e-6

# We also need to create an optimizer for updating weights
# ===============Optimizer=================                        
opt = mx.optimizer.SGD(
    learning_rate=lr,momentum=0.99,wd=0.000001)

updater = mx.optimizer.get_updater(opt)

updater.optimizer.lr_scheduler=None

In [0]:
#epoch = 0

In [147]:
epoch_r  =epoch
epoch_r
Es_train=[]
Es_valid=[]
for epoch in range(epoch_r,epoch_r+epochs+1):

    start = time.time()

    metric.reset()

    train_iter.reset()
    
    valid_iter.reset()
    

    for batch in train_iter:
        # Copy data to executor input. Note the [:].
        data[:] = batch.data[0]
        label[:] = batch.label[0]

        # Forward
        outputs=exe.forward(is_train=True)
        Es_train.append(outputs[1].asnumpy()[0])
        # Backward
        exe.backward()

        # Update
        for i, pair in enumerate(zip(exe.arg_arrays, exe.grad_arrays)):
            weight, grad = pair
            updater(i, grad, weight)   
        metric.update(batch.label[0], exe.outputs[0])#metric.update(label,p)
        
    e=metric.get()
    ed={}
    e_key,e_val = e
    for k,v in zip(e_key,e_val):
      ed[k]=v.asnumpy()[0]
    err_train=-ed['dice_coef2']
    err_train2=ed['logloss2']
    
    loss = outputs[1].asnumpy()[0] 
    
    if epoch % 10== 0:       
        #print("do_checkpoint")
        arg={k:v for k, v in arg_arrays.items() if k not in input_shapes}
        aux = dict(zip(network.list_auxiliary_states(), exe.aux_arrays))
        mx.model.save_checkpoint(model_path+prefix, epoch, network, arg, aux)
        
        
    #compute valid loss per epoch    
    metric.reset()
    for batch in valid_iter:        
        data[:] = batch.data[0]       
        label[:] = batch.label[0]
        # predict
        outputs = exe.forward(is_train=False)
        Es_valid.append(outputs[1].asnumpy()[0])
        metric.update(batch.label[0], exe.outputs[0])
    e=metric.get()
    ed={}
    e_key,e_val = e
    for k,v in zip(e_key,e_val):
      ed[k]=v.asnumpy()[0]
    err_valid=-ed['dice_coef2']
    err_valid2=ed['logloss2']
    

    end = time.time()
    print('time:',end-start,'Epoch:',epoch,"loss",loss,'trainloss:',err_train,'validloss:',err_valid,'trainloss2:',err_train2,'validloss2:',err_valid2)
    myCsvRow = [(epoch,updater.optimizer.lr,loss,err_train,err_valid,err_train2,err_valid2)]
    df = pd.DataFrame.from_records(myCsvRow,columns = col_names)
    df.to_csv(logfile, mode='a', header=False)

time: 127.34674715995789 Epoch: 0 loss -0.828995 trainloss: -0.701235 validloss: -0.625702 trainloss2: 0.148374 validloss2: 0.275829
time: 127.47548079490662 Epoch: 1 loss -0.818138 trainloss: -0.700671 validloss: -0.625982 trainloss2: 0.151173 validloss2: 0.271487
time: 127.36842203140259 Epoch: 2 loss -0.798766 trainloss: -0.699154 validloss: -0.625186 trainloss2: 0.151434 validloss2: 0.274558
time: 127.3768539428711 Epoch: 3 loss -0.829086 trainloss: -0.701596 validloss: -0.626525 trainloss2: 0.151459 validloss2: 0.277077
time: 127.30716514587402 Epoch: 4 loss -0.75531 trainloss: -0.701706 validloss: -0.627477 trainloss2: 0.149634 validloss2: 0.268524
time: 127.37158155441284 Epoch: 5 loss -0.818288 trainloss: -0.700803 validloss: -0.625203 trainloss2: 0.149884 validloss2: 0.282096
time: 127.38550424575806 Epoch: 6 loss -0.836303 trainloss: -0.701213 validloss: -0.628335 trainloss2: 0.150136 validloss2: 0.269832
time: 127.32765626907349 Epoch: 7 loss -0.770677 trainloss: -0.698557 v

In [0]:
df = pd.read_csv(logfile, index_col=0)
df.tail()

In [0]:

plt.figure()
plt.plot(df['epoch'],df['lr'])
plt.title('Learning Rate')
plt.figure()
plt.plot(df['epoch'],df['E1train'])
plt.plot(df['epoch'],df['E1valid'],'r')
plt.title('E1')
plt.figure()
plt.plot(df['epoch'],df['E2train'])
plt.plot(df['epoch'],df['E2valid'],'r')
plt.title('E2')

In [0]:
!cp logfile mode_path